# Vamos a construir un transformer desde 0 en pytorch

In [1]:
#librerias 
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math

In [17]:
Q = torch.randn(3, 3)
K = torch.randn(2, 3)

In [35]:
aux=Q.masked_fill_(torch.triu(torch.ones(3,3,dtype=bool),diagonal=1),-1e9)

In [40]:
F.softmax(aux,dim=1)

tensor([[0.3972, 0.0000, 0.0000],
        [0.1297, 0.8657, 0.0000],
        [0.4731, 0.1343, 1.0000]])

In [46]:
torch.sqrt(Q.shape[0])

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not int

### Dot product Attention

In [53]:
class dotProdAttention(nn.Module):
    def __init__(self,mask:bool = False) -> None:
        super().__init__()
        self.bool_mask=mask

    def forward(self, Q,K,V):
        #le pasamos la query, el value y la key.
        d_k=Q.shape[1]
        wi= Q.shape[0]
        wo= K.shape[0]
        aux1=torch.mm(Q,K.T) / math.sqrt(d_k)
        #si mask, entonces ponemos a -inf los que toquen
        if self.bool_mask ==True:
            aux1=aux1.masked_fill_(torch.triu(torch.ones(wi,wo,dtype=bool),diagonal=1),-1e9)
        e=F.softmax(aux1,dim=1)
        return torch.mm(e, V)

In [54]:
Q = torch.randn(4, 3)
K = torch.randn(4, 3)
V = torch.randn(4, 3)

In [55]:
a=dotProdAttention(mask=True)

In [57]:
a(Q,K,V)

tensor([[-1.4359,  1.2106, -1.1034],
        [-1.3597,  0.6453, -1.0730],
        [-1.2111,  0.1597, -0.8245],
        [-1.0443, -0.9619, -0.9149]])

### MultiHead AttentionLayer

Para esto, metemos un vector de (w, d_model) contra 3 mult de matrices, W_Q, W_K, W_V, y estos son parametros entrenables de la red. Los añadimos como parameters, o como Linear nn.
Luego de hacer las multiplicaciones, tenemos el dotAttention. Y cuando obtenemos el resultado, concatenamos y otra linear.

In [54]:
class multiHeadAttentionLayer(nn.Module):
    def __init__(self,heads,d_model) -> None:
        super().__init__()
        self.h=heads
        self.d_model=d_model
        self.d_k=int(self.d_model/self.h)
        self.d_v=int(self.d_model/self.h)

        #matrices entrenables
        #podrimos tener una de estas matrices para cada una de las cabezas, pero entonces tendrimos mucho lio
        """        
        self.W_Q=nn.Linear(self.d_model,self.d_k,bias=False)
        self.W_K=nn.Linear(self.d_model,self.d_k,bias=False)
        self.W_V=nn.Linear(self.d_model,self.d_v,bias=False)
        """
        self.W_Q=nn.Linear(self.d_model,self.d_model,bias=False)
        self.W_K=nn.Linear(self.d_model,self.d_model,bias=False)
        self.W_V=nn.Linear(self.d_model,self.d_model,bias=False)

        self.W0=nn.Linear(self.d_model,self.d_model)

    def forward(self,Q,K):#,K,V):
        #calculamos las matrices iniciales, y tenemos que separarlo en las h cabezas

        #si recibimos un tensor de forma  w x d_model, lo queremo poner de forma:  w x h x d_k
        #separando por la tercera dimension de las matrices

        #shapes
        batch_size=Q.size(0)
        leng_w=Q.size(1)

        #aplicar las matrices
        new_Q= torch.stack([torch.stack(torch.hsplit(i,self.h)) for i in self.W_Q(Q)])
        new_K=torch.stack([torch.stack(torch.hsplit(i,self.h)) for i in self.W_K(K)])
        #new_V=torch.stack([torch.stack(torch.hsplit(i,self.h)) for i in self.W_V(V)])

        #calculamos las multiplicaciones entre Q y K
        #a=torch.bmm(new_Q.view(batch_size*self.h,leng_w,self.d_k),new_K.view(batch_size*self.h,leng_w,self.d_k).transpose(1,2)).view(batch_size,self.h,leng_w,leng_w)

        #tenemos la formas, palabras (w), y d_model
        return a

In [55]:
a=multiHeadAttentionLayer(8,64)

In [56]:
Q=torch.randn(3,4,64)
K=torch.randn(3,4,64)

b=a(Q,K)

In [57]:
b.shape

torch.Size([3, 8, 4, 4])

In [117]:
c[0,0]

tensor([[ 1.3054, -0.2492, -0.3284,  0.2086, -0.0101, -0.1504,  0.1591, -0.6483],
        [ 0.8844, -0.3297, -0.2822, -0.5534, -0.1382, -0.3535,  0.1239,  0.7043],
        [-0.6641, -0.8684, -0.6955, -0.6724,  0.0105,  0.0629,  1.1549, -0.2911],
        [-0.3545,  0.2846,  0.5444, -0.7253,  0.1173,  0.1221,  0.7952,  0.0420]],
       grad_fn=<SelectBackward0>)

In [101]:
a=torch.randn(1, 4, 2)
a

tensor([[[ 0.9261,  1.2334],
         [-0.1365, -0.3900],
         [-0.2071, -0.9569],
         [-2.0184, -0.2192]]])